In [ ]:
import os, os.path
import glob
import pickle
import numpy
from numpy.polynomial import Polynomial
from scipy import ndimage, signal, interpolate
#import matplotlib
#matplotlib.use('PDF')
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014
from galpy.util import bovy_conversion, save_pickles, bovy_coords, bovy_plot
import pal5_util
from gd1_util import R0, V0
import custom_stripping_df
import seaborn as sns
import astropy.units as u
from galpy import potential
%pylab inline
from matplotlib import cm, pyplot
from __future__ import print_function
#from matplotlib.ticker import FuncFormatter, NullFormatter

def galcencyl_to_lbd(R,phi,Z,degree=True):
    xyz=bovy_coords.galcencyl_to_XYZ(R,phi,Z)
    lbd=bovy_coords.XYZ_to_lbd(xyz[0],xyz[1],xyz[2],degree=degree)
    return lbd[0], lbd[1], lbd[2]

In [ ]:
ro,vo= 8., 220.

rolr=1.375
barphi=25 #degrees


from galpy.potential import DehnenBarPotential

dp= DehnenBarPotential(rolr=rolr,barphi=barphi)

MWPotential2014bar= [MWPotential2014[0],MWPotential2014[1],MWPotential2014[2],dp]

from galpy.potential import evaluatezforces
-evaluatezforces(MWPotential2014bar, 1.,1.1/8.,phi=0.,t=1.2)*bovy_conversion.force_in_2piGmsolpc2(220.,8.)



In [ ]:
#plot potential contours

potential.plotPotentials(MWPotential2014bar,rmin=-10.*u.kpc,rmax=10.*u.kpc,zmin=-10.*u.kpc,zmax=10.*u.kpc,nrs=100,nzs=100,xy=True)



In [ ]:
print (dp.OmegaP()*(vo/ro))  #km/s/kpc
print (dp._rb*ro) # bar radius in kpc


In [ ]:
#plot orbits

ts= numpy.linspace(0,1.,1001)*u.Gyr
def plot_orbit_comparison(o):
    o.integrate(ts,potential.MWPotential2014)
    o.plot(yrange=[-2.,2.],xrange=[0.,9.],gcf=True)
    o.integrate(ts,MWPotential2014bar)
    o.plot(overplot=True,gcf=True)
    return None

def plot_orbit_comparisonXY(o):
    o.integrate(ts,potential.MWPotential2014)
    o.plot(d1='x',d2='y',xrange=[-10.,10.],yrange=[-10.,10.],gcf=True)
    o.integrate(ts,MWPotential2014bar)
    o.plot(d1='x',d2='y',overplot=True,gcf=True)
    return None


#vxvv=[R,vR,vT,z,vz,phi]

bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)
figsize(12,6)
subplot(2,2,1)
o= Orbit([1.,0.1,0.03,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparison(o)
subplot(2,2,2)
o= Orbit([1.,0.1,-0.03,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparison(o)
subplot(2,2,3)
o= Orbit([1.,0.05,0.07,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparison(o)
subplot(2,2,4)
o= Orbit([1.,0.05,-0.07,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparison(o)
tight_layout()


In [ ]:
bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)
figsize(12,6)
subplot(2,2,1)
o= Orbit([1.,0.1,0.03,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparisonXY(o)
subplot(2,2,2)
o= Orbit([1.,0.1,-0.03,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparisonXY(o)
subplot(2,2,3)
o= Orbit([1.,0.05,0.07,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparisonXY(o)
subplot(2,2,4)
o= Orbit([1.,0.05,-0.07,0.,0.1,0.],ro=ro,vo=vo)
plot_orbit_comparisonXY(o)
tight_layout()

In [ ]:
#setup Pal 5 smooth stream, only the trailing arm at the moment
sdf_trailing= pal5_util.setup_pal5model(pot=MWPotential2014SCFnobar)

#Sample N points from the smooth model today 
N=10000
R,vR,vT,z,vz,phi,dt= sdf_trailing.sample(n=N,returndt=True)
#l,b,d,vlos,pmll,pmbb = sdf_smooth.sample(n=N,lb=True)

trackRADec_trailing=bovy_coords.lb_to_radec(sdf_trailing._interpolatedObsTrackLB[:,0],
                          sdf_trailing._interpolatedObsTrackLB[:,1],
                          degree=True)

#lb_sample_trailing= sdf_trailing.sample(n=10000,lb=True)
#radec_sample_trailing= bovy_coords.lb_to_radec(lb_sample_trailing[0],
#                                               lb_sample_trailing[1],
#                                               degree=True)

In [ ]:
#add color label based on time of stripping

#stripping time bins:
t_age= 5. 
dt_bins=np.linspace(0.,t_age,6)
print (dt_bins)

l_unp=numpy.empty(N)
b_unp=numpy.empty(N)
dt_s=numpy.empty(N)

for i in range(N):
    dt_s[i]=numpy.digitize(dt[i]*bovy_conversion.time_in_Gyr(220.,8.),dt_bins)
    l_unp[i]=galcencyl_to_lbd(R[i],phi[i],z[i])[0]
    b_unp[i]=galcencyl_to_lbd(R[i],phi[i],z[i])[1]
    
radec1=bovy_coords.lb_to_radec(l_unp,b_unp,degree=True)  

In [ ]:
cmap= cm.plasma

def color_from_colormap(val,cmap,cmin,cmax):
    va=[]
    for v in val:
        va.append(cmap((v-cmin)/(cmax-cmin)))
        
    return va

def add_discrete_colorbar_dens(vmin,vmax,clabel,ticks,save_figures=False):
    fig= pyplot.gcf()
    if save_figures:
        cbar_ax = fig.add_axes([0.775,0.135,0.05,0.815])
    else:
        fig.subplots_adjust(right=0.9)
        cbar_ax = fig.add_axes([0.925, 0.13, 0.075, 0.83])
    tcmap = matplotlib.colors.ListedColormap([cmap(f) for f in numpy.linspace(0.,1.,len(ticks))])
    dtick= (ticks[1]-ticks[0])
    sm = pyplot.cm.ScalarMappable(cmap=tcmap,norm=pyplot.Normalize(vmin=vmin-0.5*dtick,vmax=vmax+0.5*dtick))
    sm._A = []
    cbar= fig.colorbar(sm,cax=cbar_ax,use_gridspec=True,format=r'$%.1f$',ticks=ticks)
    cbar.set_label(clabel)
    return None

def add_colorbar(vmin,vmax,clabel,save_figures=False):
    fig= pyplot.gcf()
    if save_figures:
        cbar_ax = fig.add_axes([0.9,0.135,0.025,0.815])
    else:
        fig.subplots_adjust(right=0.95)
        cbar_ax = fig.add_axes([0.975, 0.13, 0.025, 0.83])
    sm = pyplot.cm.ScalarMappable(cmap=cmap,norm=pyplot.Normalize(vmin=vmin,vmax=vmax))
    sm._A = []
    cbar= fig.colorbar(sm,cax=cbar_ax,use_gridspec=True,format=r'$%.1f$')
    cbar.set_label(clabel)
    return None

In [ ]:
#plot unperturbed stream 


plt.figure()
plt.scatter(radec1[:,0],radec1[:,1],s=10,color=color_from_colormap(dt_s,cmap,0.,5.))
#plt.plot(trackRADec_trailing[:,0],trackRADec_trailing[:,1],lw=2,c='r')
plt.xlabel('RA (deg)')
plt.ylabel('DEC (deg)')
add_colorbar(0.,5.,r'$t_{\rm{strip}}$ (Gyr)')
plt.show()


In [ ]:
#plot Jacobi integral to check whether it is conserved
from random import randint
ind=randint(0,len(R)-1)
print (ind)

ts= numpy.linspace(0,5.0,1001)*u.Gyr
o1= Orbit([R[ind],vR[ind],vT[ind],z[ind],vz[ind],phi[ind]],ro=ro,vo=vo)
o1.integrate(ts,MWPotential2014)
o1.plotJacobi(normed=True)

obar= Orbit([R[ind],vR[ind],vT[ind],z[ind],vz[ind],phi[ind]],ro=ro,vo=vo)
obar.integrate(ts,MWPotential2014bar)
obar.plotJacobi(normed=True)

#plot Jacobi integral w.r.t the radius 

o1.plot(d1='R',d2='Jacobi',xrange=[0.,25])

obar.plot(d1='R',d2='Jacobi',xrange=[0.,25])
axvline(dp._rb*ro,c='r') # this plots the bar radius


In [ ]:
#Integrate using scipy's odeint
#plot Jacobi integral to check whether it is conserved
#http://galpy.readthedocs.io/en/latest/reference/orbitint.html?highlight=integrate
method='odeint'

ts= numpy.linspace(0,5.0,1001)*u.Gyr
o1= Orbit([R[ind],vR[ind],vT[ind],z[ind],vz[ind],phi[ind]],ro=ro,vo=vo)
o1.integrate(ts,MWPotential2014,method=method)
o1.plotJacobi(normed=True)

obar= Orbit([R[ind],vR[ind],vT[ind],z[ind],vz[ind],phi[ind]],ro=ro,vo=vo)
obar.integrate(ts,MWPotential2014bar,method=method)
obar.plotJacobi(normed=True)

#plot Jacobi integral w.r.t the radius 

o1.plot(d1='R',d2='Jacobi',xrange=[0.,25])

obar.plot(d1='R',d2='Jacobi',xrange=[0.,25])
axvline(dp._rb*ro,c='r') # this plots the bar radius

In [ ]:
# integrate these N particles *backward* in time to when they were all stripped (they were stripped dt in the past, dt is different for each particle); something like
orbits= []
for ii in range(N):
    o= Orbit([R[ii],vR[ii],vT[ii],z[ii],vz[ii],phi[ii]]).flip() # flip flips the velocities for backwards integration
    ts= numpy.linspace(0.,dt[ii],1001)
    o.integrate(ts,MWPotential2014,method=method)
    orbits.append(o(ts[-1]).flip()) # get the orbit at the final time step, flip the velocities for forward integration below

In [ ]:
#integrate all points *forward* in time in the barred potential 
# MWPotential2014bar

finalR= numpy.empty(N)
finalphi= numpy.empty(N)
finalz= numpy.empty(N)
for ii in range(N):
    ts= numpy.linspace(0.,dt[ii],1001)
    orbits[ii].integrate(ts,MWPotential2014bar,method=method)
    finalR[ii]= orbits[ii].R(ts[-1])
    finalphi[ii]= orbits[ii].phi(ts[-1])
    finalz[ii]= orbits[ii].z(ts[-1])

In [ ]:
l_p=numpy.empty(N)
b_p=numpy.empty(N)

for i in range(N):
    l_p[i]=galcencyl_to_lbd(finalR[i],finalphi[i],finalz[i])[0]
    b_p[i]=galcencyl_to_lbd(finalR[i],finalphi[i],finalz[i])[1]
    
radec=bovy_coords.lb_to_radec(l_p,b_p,degree=True)    
    

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(radec[:,0],radec[:,1],s=2,color=color_from_colormap(dt_s,cmap,0.,5.))
plt.plot(trackRADec_trailing[:,0],trackRADec_trailing[:,1],lw=2,c='k',label='unperturbed')
plt.xlabel('RA (deg)')
plt.ylabel('DEC (deg)')
plt.legend(loc='upper left')
add_colorbar(0.,5.,r'$t_{\rm{strip}}$ (Gyr)')
#plt.savefig('Dehnenbar_effect_Pal5_rolr{}_barphi{}_intmethod_{}.pdf'.format(rolr,barphi,method))
plt.show()



In [ ]:
#compute power spectrum of the stream affected by the bar
mxieta= pal5_util.radec_to_pal5xieta(radec[:,0],radec[:,1],degree=True)
xi= mxieta[:,0]
eta=mxieta[:,1]

In [ ]:
#write xi eta to a file 

fo=open('pal5_DehnenBar_rolr1p375_barphi25_10000samples.dat','w')
fo.write("#xi   eta" + "\n")
for jj in range(len(xi)):
        fo.write(str(xi[jj]) + str(eta[jj]) + "\n")
    
fo.close()

In [ ]:
#load computed sampled points



In [ ]:
from sklearn.neighbors import KernelDensity
xxs= numpy.linspace(minxi,maxxi,1000)[:, numpy.newaxis]
xxi=xi[:, numpy.newaxis]
kde = KernelDensity(kernel='epanechnikov', bandwidth=0.7).fit(xxi)
log_dens = kde.score_samples(xxs)

plt.figure()
plt.plot(xxs[:, 0], numpy.exp(log_dens),lw=2)
plt.xlabel(r'$\xi$ (deg)')
plt.ylabel('dens contrast')
plt.show()

In [ ]:
minxi = 0.25
maxxi= 14.35
bw= 0.1

nbins= (maxxi - minxi)/bw
print (nbins)

xibin_edge=[(minxi-0.5*bw) + i*bw for i in range(int(nbins)+2)]

#print xibin_edge

midxi=[0.5*(xibin_edge[i] + xibin_edge[i+1]) for i in range(int(nbins)+1)]
#print len(midxi)

nden=np.histogram(xi,xibin_edge,density=True)[0]
Ndn=np.histogram(xi,xibin_edge)[0]

shot_noise=numpy.sqrt(Ndn)


bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)
figsize(12,10.)
xrange=[0.,14.5]
#hist(xi,xibin_edge,normed=True)
plot(midxi,nden,ls='steps-mid')
errorbar(midxi,nden,shot_noise/(N*bw),marker='None',ls='None')

In [ ]:
#load the saved pickle file for converting apars to obs coords
with open('pal5_64sampling.pkl','rb') as savefile: #cached
            sdf_smooth= pickle.load(savefile)
            sdf_pepper= pickle.load(savefile)
            

def convert_dens_to_obs(apars,dens,dens_smooth,mO):
    """
    NAME:
        convert_dens_to_obs
    PURPOSE:
        Convert track to observed coordinates
    INPUT:
        apars - parallel angles
        dens - density(apars)
        dens_smooth - smooth-stream density(apars)
        mO= (None) mean parallel frequency (1D) 
            [needs to be set to get density on same grid as track]
        poly_deg= (3) degree of the polynomial to fit for the 'smooth' stream
    OUTPUT:
        (xi,dens/smooth)
    """
    mT= sdf_pepper.meanTrack(apars,_mO=mO,coord='lb')
    mradec= bovy_coords.lb_to_radec(mT[0],mT[1],degree=True)
    mxieta= pal5_util.radec_to_pal5xieta(mradec[:,0],mradec[:,1],degree=True)
    outll= numpy.arange(minxi,14.35,0.1)
    # Interpolate density
    ipll= interpolate.InterpolatedUnivariateSpline(mxieta[:,0],apars)
    ipdens= interpolate.InterpolatedUnivariateSpline(apars,dens/dens_smooth)
    return (outll,ipdens(ipll(outll)))
            



def compute_Pk(filename, scaling = 'spectrum'):
    data= numpy.loadtxt(filename)
    p1=data[:,0]
    tdata=data[:,1]
    data_err=data[:,2]
    
    px,py= signal.csd(tdata,tdata,fs=1./(p1[1]-p1[0]),scaling=scaling,nperseg=len(p1))
    py= py.real
    px= 1./px
    py= numpy.sqrt(py*(p1[-1]-p1[0]))
    
            
    # Perform simulations of the noise to determine the power in the noise
    nerrsim= 10000
    ppy_err= numpy.empty((nerrsim,len(px)))
    for ii in range(nerrsim):
        tmock= data_err*numpy.random.normal(size=len(p1))
        ppy_err[ii]= signal.csd(tmock,tmock,
                                fs=1./(p1[1]-p1[0]),scaling=scaling,
                                nperseg=len(p1))[1].real
    py_err= numpy.sqrt(numpy.median(ppy_err,axis=0)*(p1[-1]-p1[0]))
    
    #return 180./px/numpy.pi, py, py_err #if apars
    return px, py, py_err




def set_ranges_and_labels_dens():
    bovy_plot.bovy_text(r'$\sqrt{\delta\delta}$',top_left=True,size=18.)
    xlabel(r'$1/k_{\phi_{1}}\,(\mathrm{deg})$',size=18)
    ylim(0.001,10.)
    xlim(1.,150.)
    for axis in [gca().xaxis,gca().yaxis]:
        axis.set_major_formatter(FuncFormatter(
                lambda y,pos: (r'${{:.{:1d}f}}$'.format(int(numpy.maximum(-numpy.log10(y),0)))).format(y)))
    return None